In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# show all columns 
pd.set_option('display.max_columns', None)
# show entire rows
pd.set_option('display.max_rows', None)
# show all values in a cell
pd.set_option('display.max_colwidth', None)

In [36]:
# import dataset
df_clean = pd.read_excel('../Data/clean_data_sp_w_revenue_w_potential.xlsx')
df_raw = pd.read_excel('../Data/raw_data.xlsx')

df = pd.merge(df_clean, df_raw, on='nome', how='inner', suffixes=('', '_raw'))
df=df[['codigo']+df_clean.columns.to_list()]

In [34]:
df.head()

,codigo,nome,população,popAte9,popDe10a14,popDe15a19,popDe20a24,popDe25a34,popDe35a49,popDe50a59,popMaisDe60,domiciliosA1,domiciliosA2,domiciliosB1,domiciliosB2,domiciliosC1,domiciliosC2,domiciliosD,domiciliosE,rendaMedia,faturamento,potencial,popDe25a49,domiciliosA,domiciliosB,popAte14,domicilios,propDomiciliosCriancas,propDomiciliosIdosos,rendaTotal,rendaMediaPop,propDomiciliosA,propDomiciliosB,propDomiciliosC,propDomiciliosD,propDomiciliosE
0,355030251,A. E. Carvalho,94034,12668,6853,9836,7487,14535,21549,10598,10508,0,253,2197,4368,6681,7011,2247,5670,1501,4.970912e+05,Alto,36084,253,6565,19521,28427,0.686706,0.369649,42668927,453.760629,0.008900,0.230942,0.481655,0.079045,0.199458
1,35503020,Aclimação,32791,2297,1017,2096,2197,5341,7281,4917,7645,1413,1734,3704,2351,1946,827,291,1617,5920,1.332529e+06,Alto,12622,3147,6055,3314,13883,0.238709,0.550673,82187360,2506.399927,0.226680,0.436145,0.199741,0.020961,0.116473
2,355030285,Adventista,104193,15070,7343,10631,8657,17749,23364,11567,9812,0,0,1423,4875,8595,10082,3111,5776,1284,1.887401e+05,Médio,41113,0,6298,22413,33862,0.661892,0.289764,43478808,417.291066,0.000000,0.185990,0.551562,0.091873,0.170575
3,35503088,Água Branca,12721,953,343,627,819,2142,2833,1790,3214,624,667,1558,1032,915,361,84,404,6278,1.181028e+06,Alto,4975,1291,2590,1296,5645,0.229584,0.569353,35439310,2785.890260,0.228698,0.458813,0.226041,0.014880,0.071568
4,35503066,Água Funda,48417,5078,2396,4018,3571,7388,10751,6648,8567,0,303,1794,2986,4489,2836,1104,2553,1905,7.844942e+05,Alto,18139,303,4780,7474,16065,0.465235,0.533271,30603825,632.088419,0.018861,0.297541,0.455960,0.068721,0.158917


In [50]:
# agrpamento por faixas de renda
# crie uma faixa com 5 valores de renda
df['faixa_renda'] = pd.qcut(df['rendaMedia'], 5)
df['faixa_etaria_alvo'] = pd.qcut(df['popDe25a49'], 5)
df['faixa_domiciliosA'] = pd.qcut(df['domiciliosA'], 5)
df['faixa_domiciliosB'] = pd.qcut(df['domiciliosB'], 5)
df['faixa_codigo'] = pd.qcut(df['codigo'], 5)

agrupamento = df.groupby(['faixa_renda','faixa_etaria_alvo','faixa_domiciliosA','faixa_domiciliosB']).median()
# omitir NaN da coluna 'codigo'
agrupamento = agrupamento.dropna(subset=['codigo'])
# faixa de renda A = '(6577.6, 21677.0]' e B = '(3254.4, 6577.6]'
# rendaMedia entre 3254.4 e 21677.0
agrupamento = agrupamento[agrupamento['rendaMedia'] >= 3254.4]
agrupamento

/tmp/ipykernel_2777/3241792591.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  agrupamento = df.groupby(['faixa_renda','faixa_etaria_alvo','faixa_domiciliosA','faixa_domiciliosB']).median()


codigo  \
faixa_renda       faixa_etaria_alvo  faixa_domiciliosA faixa_domiciliosB                
(3254.4, 6577.6]  (13.999, 4864.2]   (1.4, 191.0]      (5.999, 1597.6]    355030137.5   
                                     (191.0, 698.2]    (5.999, 1597.6]     35503047.0   
                                                       (1597.6, 3359.6]   355030202.0   
                                     (698.2, 1768.0]   (1597.6, 3359.6]   355030135.0   
                  (4864.2, 9655.8]   (191.0, 698.2]    (1597.6, 3359.6]    35503017.0   
                                                       (3359.6, 4509.0]     3550304.0   
                                     (698.2, 1768.0]   (5.999, 1597.6]    355030214.0   
                                                       (1597.6, 3359.6]    35503088.0   
                                                       (3359.6, 4509.0]    35503052.0   
                                     (1768.0, 12344.0] (3359.6, 4509.0]    35503069.0   
                  (9655.8, 15609.2]  (698.2, 1768.0]   (3359.6, 4509.0]    35503036.0   
                                                       (4509.0, 6427.2]    35503047.5   
                                     (1768.0, 12344.0] (4509.0, 6427.2]    35503080.5   
                                                       (6427.2, 16930.0]   35503062.0   
                  (15609.2, 24167.2] (698.2, 1768.0]   (6427.2, 16930.0]  355030116.0   
                                     (1768.0, 12344.0] (4509.0, 6427.2]   355030224.0   
                                                       (6427.2, 16930.0]  355030106.0   
                  (24167.2, 87183.0] (1768.0, 12344.0] (6427.2, 16930.0]  355030126.0   
(6577.6, 21677.0] (13.999, 4864.2]   (191.0, 698.2]    (5.999, 1597.6]    355030210.0   
                                     (698.2, 1768.0]   (5.999, 1597.6]    355030136.0   
                                                       (1597.6, 3359.6]    35503076.0   
                                     (1768.0, 12344.0] (5.999, 1597.6]     35503073.0   
                                                       (1597.6, 3359.6]   355030172.5   
                  (4864.2, 9655.8]   (698.2, 1768.0]   (1597.6, 3359.6]   355030137.0   
                                                       (3359.6, 4509.0]   355030144.0   
                                     (1768.0, 12344.0] (5.999, 1597.6]    355030211.0   
                                                       (1597.6, 3359.6]   355030143.0   
                                                       (3359.6, 4509.0]   355030134.5   
                                                       (4509.0, 6427.2]    35503049.0   
                  (9655.8, 15609.2]  (698.2, 1768.0]   (3359.6, 4509.0]   355030276.0   
                                     (1768.0, 12344.0] (3359.6, 4509.0]   355030207.0   
                                                       (4509.0, 6427.2]    35503080.0   
                                                       (6427.2, 16930.0]   35503070.0   
                  (15609.2, 24167.2] (1768.0, 12344.0] (3359.6, 4509.0]   355030206.0   
                                                       (6427.2, 16930.0]  195266593.0   
                  (24167.2, 87183.0] (1768.0, 12344.0] (6427.2, 16930.0]   35503072.0   

                                                                          população  \
faixa_renda       faixa_etaria_alvo  faixa_domiciliosA faixa_domiciliosB              
(3254.4, 6577.6]  (13.999, 4864.2]   (1.4, 191.0]      (5.999, 1597.6]       2462.5   
                                     (191.0, 698.2]    (5.999, 1597.6]       7786.0   
                                                       (1597.6, 3359.6]      8991.0   
                                     (698.2, 1768.0]   (1597.6, 3359.6]     10376.0   
                  (4864.2, 9655.8]   (191.0, 698.2]    (1597.6, 3359.6]     18608.0   
                                                       (3359.6, 4509.0]     14861.0   


In [45]:
df[df['codigo']==35503072.0]

,codigo,nome,população,popAte9,popDe10a14,popDe15a19,popDe20a24,popDe25a34,popDe35a49,popDe50a59,popMaisDe60,domiciliosA1,domiciliosA2,domiciliosB1,domiciliosB2,domiciliosC1,domiciliosC2,domiciliosD,domiciliosE,rendaMedia,faturamento,potencial,popDe25a49,domiciliosA,domiciliosB,popAte14,domicilios,propDomiciliosCriancas,propDomiciliosIdosos,rendaTotal,rendaMediaPop,propDomiciliosA,propDomiciliosB,propDomiciliosC,propDomiciliosD,propDomiciliosE,faixa_renda,faixa_etaria_alvo,faixa_domiciliosA,faixa_domiciliosB,faixa_codigo
162,35503072,Moema,60709,4607,1757,3051,3144,9882,14562,9801,13905,6982,5362,6631,2481,1881,736,0,3719,11332,2.058364e+06,Alto,24444,12344,9112,6364,27792,0.228987,0.500324,314938944,5187.681299,0.444157,0.327864,0.094164,0.0,0.133815,"(6577.6, 21677.0]","(24167.2, 87183.0]","(1768.0, 12344.0]","(6427.2, 16930.0]","(35503062.2, 355030123.4]"
